In [2]:
# import

import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

pd.set_option('display.max_rows', 300) # specifies number of rows to show
pd.options.display.float_format = '{:40,.4f}'.format # specifies default number format to 4 decimal places
plt.style.use('ggplot') # specifies that graphs should use ggplot styling

0.13.5


In [6]:
# read csv

data = pd.read_csv(r"D:\UCL_CODE\CASA0007\data\Seattle\Seattle_listings.csv")

data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5902 entries, 0 to 5901
Data columns (total 66 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5902 non-null   int64  
 1   listing_url                                   5902 non-null   object 
 2   scrape_id                                     5902 non-null   int64  
 3   last_scraped                                  5902 non-null   object 
 4   source                                        5902 non-null   object 
 5   host_id                                       5902 non-null   int64  
 6   host_since                                    5902 non-null   object 
 7   host_location                                 5902 non-null   object 
 8   host_response_time                            5902 non-null   object 
 9   host_response_rate                            5902 non-null   i

,id,listing_url,scrape_id,last_scraped,source,host_id,host_since,host_location,host_response_time,host_response_rate,...,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,license_num
0,53939356,https://www.airbnb.com/rooms/53939356,20220918143902,2022-09-18,city scrape,435279521,2021-12-08,"Lake Forest Park, WA",within an hour,100,...,4.3300,4.4400,0,t,7,0,7,0,2.5200,0
1,630392837253856994,https://www.airbnb.com/rooms/630392837253856994,20220918143902,2022-09-18,city scrape,104834083,2016-11-22,,within an hour,98,...,4.5000,4.5000,0,t,46,46,0,0,1.5000,0
2,19553441,https://www.airbnb.com/rooms/19553441,20220918143902,2022-09-18,city scrape,25383292,2014-12-30,"Seattle, WA",No response,0,...,5.0000,4.9000,STR-OPLI-20-000201,f,2,2,0,0,0.3200,1
3,599272579997904543,https://www.airbnb.com/rooms/599272579997904543,20220918143902,2022-09-18,city scrape,33439044,2015-05-15,"Seattle, WA",within an hour,100,...,0.0000,NaN,0,t,2,2,0,0,0.0000,0
4,606740310546802034,https://www.airbnb.com/rooms/606740310546802034,20220918143902,2022-09-18,city scrape,121968404,2017-03-22,"Seattle, WA",within an hour,100,...,4.8600,5.0000,0,f,1,1,0,0,2.4700,0


In [7]:
# data preparation

data['response_time_num'] = 0
for i in range(len(data)):
    if data['host_response_time'].iloc[i] == 'No response':
        data['response_time_num'].iloc[i] = 1
    elif data['host_response_time'].iloc[i] == 'a few days or more':
        data['response_time_num'].iloc[i] = 2
    elif data['host_response_time'].iloc[i] == 'within a day':
        data['response_time_num'].iloc[i] = 3
    elif data['host_response_time'].iloc[i] == 'within a few hours':
        data['response_time_num'].iloc[i] = 4
    elif data['host_response_time'].iloc[i] == 'within an hour':
        data['response_time_num'].iloc[i] = 5

data['instant_bookable_num'] = 0
for i in range(len(data)):
    if data['instant_bookable'].iloc[i] == 't':
        data['instant_bookable_num'].iloc[i] = 1
        
data['superhost_num'] = 0
for i in range(len(data)):
    if data['host_is_superhost'].iloc[i] == 't':
        data['superhost_num'].iloc[i] = 1

data = pd.get_dummies(data, columns=['source'])
data = pd.get_dummies(data, columns=['room_type'])

data.info()
data.head()

C:\Users\xianl\AppData\Local\Temp\ipykernel_42884\534545328.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['response_time_num'].iloc[i] = 5
C:\Users\xianl\AppData\Local\Temp\ipykernel_42884\534545328.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['response_time_num'].iloc[i] = 1
C:\Users\xianl\AppData\Local\Temp\ipykernel_42884\534545328.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['response_time_num']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5902 entries, 0 to 5901
Data columns (total 72 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5902 non-null   int64  
 1   listing_url                                   5902 non-null   object 
 2   scrape_id                                     5902 non-null   int64  
 3   last_scraped                                  5902 non-null   object 
 4   host_id                                       5902 non-null   int64  
 5   host_since                                    5902 non-null   object 
 6   host_location                                 5902 non-null   object 
 7   host_response_time                            5902 non-null   object 
 8   host_response_rate                            5902 non-null   int64  
 9   host_acceptance_rate                          5902 non-null   f

,id,listing_url,scrape_id,last_scraped,host_id,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,...,reviews_per_month,license_num,response_time_num,instant_bookable_num,superhost_num,source_city scrape,source_previous scrape,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,53939356,https://www.airbnb.com/rooms/53939356,20220918143902,2022-09-18,435279521,2021-12-08,"Lake Forest Park, WA",within an hour,100,100.0000,...,2.5200,0,5,1,0,1,0,0,1,0
1,630392837253856994,https://www.airbnb.com/rooms/630392837253856994,20220918143902,2022-09-18,104834083,2016-11-22,,within an hour,98,99.0000,...,1.5000,0,5,1,0,1,0,1,0,0
2,19553441,https://www.airbnb.com/rooms/19553441,20220918143902,2022-09-18,25383292,2014-12-30,"Seattle, WA",No response,0,33.0000,...,0.3200,1,1,0,1,1,0,1,0,0
3,599272579997904543,https://www.airbnb.com/rooms/599272579997904543,20220918143902,2022-09-18,33439044,2015-05-15,"Seattle, WA",within an hour,100,0.0000,...,0.0000,0,5,1,0,1,0,1,0,0
4,606740310546802034,https://www.airbnb.com/rooms/606740310546802034,20220918143902,2022-09-18,121968404,2017-03-22,"Seattle, WA",within an hour,100,100.0000,...,2.4700,0,5,0,0,1,0,1,0,0


In [9]:
## data select

data_final = data.drop(['listing_url', 'scrape_id', 'last_scraped', 'host_id', 'host_location', 'host_response_time'], axis=1)
data_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5902 entries, 0 to 5901
Data columns (total 66 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            5902 non-null   int64  
 1   host_since                                    5902 non-null   object 
 2   host_response_rate                            5902 non-null   int64  
 3   host_acceptance_rate                          5902 non-null   float64
 4   host_is_superhost                             5902 non-null   object 
 5   host_neighbourhood                            5902 non-null   object 
 6   host_listings_count                           5902 non-null   int64  
 7   host_total_listings_count                     5902 non-null   int64  
 8   host_verifications                            5902 non-null   object 
 9   host_has_profile_pic                          5902 non-null   o